In [ ]:
# 기준size (512,512,1)

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/')
'/content/drive/My Drive/Study/modelCheckpoint/____________________k46_boston_{epoch:02d}-{val_loss:.4f}.hdf5'

Mounted at /content/drive/


'/content/drive/My Drive/Study/modelCheckpoint/____________________k46_boston_{epoch:02d}-{val_loss:.4f}.hdf5'

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
# from selenium.webdriver.common.keys import Keys

In [3]:
def make_mosaic(X,ratio=12):
    newX = np.zeros((np.array(X.shape)).astype(int))
    kernel = np.ones((ratio,ratio))
    
    for k in range(int(X.shape[0]/4),int(X.shape[0]/4)*3,ratio):
        for l in range(int(X.shape[1]/4),int(X.shape[1]/4)*3,ratio):
            mean = np.mean(X[k:k+ratio,l:l+ratio]*kernel)
            newX[k:k+ratio,l:l+ratio] = mean

    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            if int(X.shape[0]/4)<=i<3*int(X.shape[0]/4) and int(X.shape[1]/4)<=j<3*int(X.shape[1]/4):
                continue
            else:
                newX[i,j] = X[i,j]
    
    return newX

In [ ]:
npy_x = np.load("/content/drive/My Drive/P_project/"+"datasets/npy/npy_x_128_4.npy")/255.0
npy_y = np.load("/content/drive/My Drive/P_project/"+"datasets/npy/npy_y_128_4.npy")[:,int(npy_x.shape[1]/4):3*int(npy_x.shape[1]/4),int(npy_x.shape[2]/4):3*int(npy_x.shape[2]/4),:]
npy_x.shape,npy_y.shape

((17659, 128, 128, 1), (17659, 64, 64, 1))

In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dense,Input,concatenate,Reshape,Flatten
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
optimizer = Adam(lr=0.002)
es = EarlyStopping(monitor = "val_loss",patience=30)
reLR = ReduceLROnPlateau(monitor = 'val_loss',patience=14)

for k in range(100,200):
  modelpath = "/content/drive/My Drive/P_project/datasets/models/{}.h5".format(k)
  cp = ModelCheckpoint(monitor = 'val_loss',filepath = modelpath,save_best_only=True)
  inputs = Input(shape=(128,128,1))
  x = Flatten()(inputs)
  x = Dense(64*64,activation='relu')(x)
  outputs = Reshape((64,64,1))(x)
  model = Model(inputs = inputs,outputs=outputs)
  model.compile(loss = "mse",optimizer = optimizer)
  model.fit(npy_x,npy_y,epochs=500,validation_split=0.15,batch_size=32,callbacks=[cp,es,reLR])


Epoch 1/500
470/470 [==============================] - 201s 427ms/step - loss: 5026.1682 - val_loss: 2759.3750
Epoch 2/500
470/470 [==============================] - 184s 391ms/step - loss: 2215.7347 - val_loss: 2264.1135
Epoch 3/500
470/470 [==============================] - 183s 390ms/step - loss: 1877.7751 - val_loss: 2020.2122
Epoch 4/500
470/470 [==============================] - 183s 390ms/step - loss: 1694.2002 - val_loss: 1861.4852
Epoch 5/500
470/470 [==============================] - 183s 389ms/step - loss: 1578.1204 - val_loss: 1753.0577
Epoch 6/500
470/470 [==============================] - 183s 389ms/step - loss: 1475.7872 - val_loss: 1659.9156
Epoch 7/500
470/470 [==============================] - 182s 388ms/step - loss: 1405.4911 - val_loss: 1622.7208
Epoch 8/500
470/470 [==============================] - 182s 387ms/step - loss: 1375.2255 - val_loss: 1542.2394
Epoch 9/500
470/470 [==============================] - 184s 391ms/step - loss: 1314.4783 - val_loss: 1486.6547
E